In [1]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
from transformers import GPTJForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, default_data_collator
from datasets import load_dataset
import time
import torch
import os
import numpy as np
import evaluate
import sklearn
import pandas as pd
import ray
import ray.data
from ray.data.preprocessors import BatchMapper, Chain
import os
#os.environ["RAY_ML_DEV"] = "1"



ray.init(runtime_env={"env_vars": {"NCCL_SOCKET_IFNAME": "ens5"}})
start = time.time()
name = "gpt-j-6B"

comet_ml is installed but `COMET_API_KEY` is not set.
/home/ray/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
2023-02-09 14:50:46,570	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 10.0.48.108:6379...
2023-02-09 14:50:46,580	INFO worker.py:1544 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8265 
2023-02-09 14:50:46,583	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_1e3dac5337413e0660dd7da00d1f0b6e.zip' (0.13MiB) to Ray cluster...
2023-02-09 14:50:46,585	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_1e3dac5337413e0660dd7da00d1f0b6e.zip'.


In [2]:
print("Loading dataset")
# current_dataset = load_dataset("wikitext", 'wikitext-103-v1', split="train")
current_dataset = load_dataset("tiny_shakespeare")
current_dataset

Loading dataset


Found cached dataset tiny_shakespeare (/home/ray/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1
    })
})

In [3]:
if isinstance(current_dataset, dict):
    ray_datasets = ray.data.from_huggingface(current_dataset)
else:
    ray_dataset: ray.data.Dataset = ray.data.from_huggingface(current_dataset)
    train, validation, test = ray_dataset.random_shuffle(seed=1).split_proportionately([0.9])
    ray_datasets = {"train": train.repartition(16), "validation": validation.repartition(4)}

In [4]:
block_size = 1024
def split_column_with_one_string(df):
    data = df["text"].iloc[0]
    df = pd.DataFrame()
    #df["text"] = [x.strip() for x in data.split("\n\n") if x.strip()]
    df["text"] = [data[i:i+block_size].strip() for i in range(0, len(data), block_size)]
    return df

string_splitter = BatchMapper(split_column_with_one_string, batch_format="pandas")

In [5]:
from ray.data.preprocessor import Preprocessor

class Tokenizer:
    def __init__(self, pretrained_model_name_or_path, caption_column, revision=None) -> None:
        # Importing here to work around a memory leak with Ray Data in 2.2
        # Should be fixed in 2.3 or 2.4
        from transformers import AutoTokenizer

        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path, revision=revision)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.caption_column = caption_column

    # Preprocessing the datasets.
    # We need to tokenize input captions and transform the images.
    def tokenize_captions(self, txt_list, is_train=True):
        tokens = self.tokenizer(list(txt_list[self.caption_column]), truncation=True,
                                       max_length=self.tokenizer.model_max_length, padding="max_length",return_tensors="np",)        
        tokens["labels"] = tokens["input_ids"].copy()
        return {k: v for k, v in tokens.items()}

    def __call__(self, df: "pd.DataFrame") -> "pd.DataFrame":
        return self.tokenize_captions(df)


class TokenizerPreprocessor(Preprocessor):
    _is_fittable = False

    def __init__(self, pretrained_model_name_or_path, caption_column, revision=None) -> None:
        self.pretrained_model_name_or_path = pretrained_model_name_or_path
        self.revision = revision
        self.caption_column = caption_column

    _transform_pandas = Tokenizer

    def _get_transform_config(self):
        """Returns kwargs to be passed to :meth:`ray.data.Dataset.map_batches`.
        This can be implemented by subclassing preprocessors.
        """
        return dict(
            compute=ray.data.ActorPoolStrategy(),
            fn_constructor_kwargs=dict(
                pretrained_model_name_or_path=self.pretrained_model_name_or_path,
                revision=self.revision,
                caption_column=self.caption_column,
            ),
        )


In [86]:
from transformers.utils.logging import disable_progress_bar, enable_progress_bar
from transformers.utils.hub import cached_file
from accelerate.big_modeling import get_balanced_memory, infer_auto_device_map, load_checkpoint_and_dispatch, init_empty_weights
from deepspeed.ops.adam.cpu_adam import DeepSpeedCPUAdam
from ray.air import session
import torch
import os
from transformers.trainer_pt_utils import get_parameter_names
from torch import nn


num_cpus = 8

class TrainingArgumentsPatched(TrainingArguments):
    @property
    def place_model_on_device(self):
        return False

def trainer_init_per_worker(train_dataset, eval_dataset = None, **config):
    # Env vars necessary for HF to setup DDP
    #os.environ.pop("RANK")
    #os.environ.pop("WORLD_SIZE")
    #os.environ.pop("LOCAL_RANK")

    os.environ["OMP_NUM_THREADS"] = str(num_cpus)
    torch.backends.cuda.matmul.allow_tf32 = True

    batch_size = 6
    deepspeed = {
        "fp16": {
            "enabled": "auto",
            "initial_scale_power": 32,
        },
        "bf16":{
            "enabled":"auto"
        },    
        "optimizer": {
            "type": "AdamW",
            "params": {
            "lr": "auto",
            "betas": "auto",
            "eps":"auto",
            }
        },
        "zero_optimization": {
            "stage": 3,
            "offload_optimizer": {
                "device": "cpu",
                "pin_memory": False,
            },
           # "offload_param": {
           #     "device": "cpu",
           #     "pin_memory": False,
           # },
            "overlap_comm": True,
            "contiguous_gradients": True,
            "reduce_bucket_size": "auto",
            "stage3_prefetch_bucket_size": "auto",
            "stage3_param_persistence_threshold": "auto",
            "gather_16bit_weights_on_model_save": True,
            "round_robin_gradients": True,
        },
        "gradient_accumulation_steps": "auto",
        "gradient_clipping": "auto",
        "steps_per_print": 1,
        "train_batch_size": "auto",
        "train_micro_batch_size_per_gpu": "auto",
        "wall_clock_breakdown": False
    }
    print("Preparing training arguments")
    training_args = TrainingArguments(
        name,
        per_device_train_batch_size=batch_size,
        logging_steps=1, save_strategy="steps",
        save_steps=490,
        per_device_eval_batch_size=batch_size,
        learning_rate=0.001,
        weight_decay=0.01,
        # warmup_steps=20,
        label_names=['input_ids', 'attention_mask'],  # 'logits', 'past_key_values'
        num_train_epochs=config.get("epochs", 2),
        push_to_hub=False,
        disable_tqdm=True,  # declutter the output a little
        bf16=True,
        gradient_checkpointing=True,
        #local_rank=-1,
        deepspeed=deepspeed
    )
    disable_progress_bar()
    dtype = torch.float32

    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
    tokenizer.pad_token = tokenizer.eos_token

    print("Loading model")

    #with init_empty_weights():
    model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", use_cache=False)
    model.resize_token_embeddings(len(tokenizer))

    decay_parameters = get_parameter_names(model, [nn.LayerNorm])
    decay_parameters = [name for name in decay_parameters if "bias" not in name]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if n in decay_parameters],
            "weight_decay": training_args.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
            "weight_decay": 0.0,
        },
    ]

    optimizer = SM3(
        optimizer_grouped_parameters,
        beta=0,
        eps=training_args.adam_epsilon,
        lr=training_args.learning_rate,
    )

    # # Load the checkpoint and dispatch it to the right devices
    # model = load_checkpoint_and_dispatch(
    #     model,
    #     cached_file("EleutherAI/gpt-j-6B", "pytorch_model.bin", revision="float16", local_files_only=True),
    #     device_map="auto",
    #     no_split_module_classes=["GPTJBlock"],
    #     dtype=dtype,
    # )

    enable_progress_bar()

    metric = evaluate.load("accuracy")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        #optimizers=(optimizer, None)
    )
    return trainer

In [87]:
from ray.train.huggingface import HuggingFaceTrainer
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.air.integrations.mlflow import MLflowLoggerCallback
from ray.tune import SyncConfig

class HuggingFaceTrainerPatched(HuggingFaceTrainer):
    def _validate_attributes(self):
        for key, conf in self._dataset_config.items():
            if conf.use_stream_api:
                raise ValueError(
                    "HuggingFaceTrainer does not support `use_stream_api`."
                )
        super(HuggingFaceTrainer, self)._validate_attributes()

trainer = HuggingFaceTrainerPatched(
    trainer_init_per_worker=trainer_init_per_worker,
    scaling_config=ScalingConfig(num_workers=16, use_gpu=True, resources_per_worker={"GPU": 1, "CPU": 96/8}),
    datasets={"train": ray_datasets["train"], "evaluation": ray_datasets["validation"]},
    run_config=RunConfig(
        local_dir="/mnt/cluster_storage/",
        sync_config=SyncConfig(syncer=None),
        callbacks=[MLflowLoggerCallback(experiment_name=name)],
        checkpoint_config=CheckpointConfig(num_to_keep=1, checkpoint_score_attribute="eval_loss", checkpoint_score_order="min"),
    ),
    preprocessor=Chain(string_splitter, TokenizerPreprocessor("EleutherAI/gpt-j-6B", "text")),
)

In [88]:
results = trainer.fit()

2023-02-09 16:54:45,129	WARNING trial_runner.py:369 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (440 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.
(pid=74513, ip=10.0.55.91) /home/ray/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(pid=74513, ip=10.0.55.91)   from pandas import MultiIndex, Int64Index
(pid=74513, ip=10.0.55.91) comet_ml is installed but `COMET_API_KEY` is not set.
(HuggingFaceTrainerPatched pid=74513, ip=10.0.55.91) 2023-02-09 16:54:57,638	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> AllToAllOper

(RayTrainWorker pid=66668, ip=10.0.15.54) Preparing training arguments


(RayTrainWorker pid=66675, ip=10.0.15.54) /home/ray/anaconda3/lib/python3.8/site-packages/ray/data/_internal/bulk_dataset_iterator.py:108: UserWarning: session.get_dataset_shard returns a ray.data.DatasetIterator instead of a Dataset as of Ray v2.3. Use iter_torch_batches(), to_tf(), or iter_batches() to iterate over one epoch. See https://docs.ray.io/en/latest/data/api/dataset_iterator.html for full DatasetIterator docs.
(RayTrainWorker pid=66675, ip=10.0.15.54)   warnings.warn(


(RayTrainWorker pid=66675, ip=10.0.15.54) Preparing training arguments
(RayTrainWorker pid=66679, ip=10.0.15.54) Preparing training arguments


(RayTrainWorker pid=66679, ip=10.0.15.54) /home/ray/anaconda3/lib/python3.8/site-packages/ray/data/_internal/bulk_dataset_iterator.py:108: UserWarning: session.get_dataset_shard returns a ray.data.DatasetIterator instead of a Dataset as of Ray v2.3. Use iter_torch_batches(), to_tf(), or iter_batches() to iterate over one epoch. See https://docs.ray.io/en/latest/data/api/dataset_iterator.html for full DatasetIterator docs.
(RayTrainWorker pid=66679, ip=10.0.15.54)   warnings.warn(


...

(RayTrainWorker pid=66667, ip=10.0.15.54) {'loss': 10.3281, 'learning_rate': 0.0009545454545454546, 'epoch': 0.09}
(RayTrainWorker pid=75470, ip=10.0.55.91) [2023-02-09 16:57:13,309] [INFO] [logging.py:68:log_dist] [Rank 0] step=1, skipped=0, lr=[0.0009545454545454546], mom=[[0.9, 0.999]]
(RayTrainWorker pid=75470, ip=10.0.55.91) {'loss': 10.3281, 'learning_rate': 0.0009545454545454546, 'epoch': 0.09}
(RayTrainWorker pid=66667, ip=10.0.15.54) {'loss': 7.8867, 'learning_rate': 0.0009090909090909091, 'epoch': 0.18}
(RayTrainWorker pid=75470, ip=10.0.55.91) [2023-02-09 16:57:46,744] [WARNING] [stage3.py:1939:step] 2 pytorch allocator cache flushes since last step. this happens when there is high memory pressure and is detrimental to performance. if this is happening frequently consider adjusting settings to reduce memory consumption. If you are unable to make the cache flushes go away consider adding torch.cuda.empty_cache() calls in your training loop to ensure that all ranks flush their

(RayTrainWorker pid=75470, ip=10.0.55.91) Saving model checkpoint to gpt-j-6B/checkpoint-22
(RayTrainWorker pid=75470, ip=10.0.55.91) Configuration saved in gpt-j-6B/checkpoint-22/config.json
(RayTrainWorker pid=75470, ip=10.0.55.91) Configuration saved in gpt-j-6B/checkpoint-22/generation_config.json


(RayTrainWorker pid=66667, ip=10.0.15.54) {'loss': 0.9338, 'learning_rate': 0.0, 'epoch': 2.0}


(RayTrainWorker pid=75470, ip=10.0.55.91) Model weights saved in gpt-j-6B/checkpoint-22/pytorch_model.bin
(RayTrainWorker pid=75470, ip=10.0.55.91) tokenizer config file saved in gpt-j-6B/checkpoint-22/tokenizer_config.json
(RayTrainWorker pid=75470, ip=10.0.55.91) Special tokens file saved in gpt-j-6B/checkpoint-22/special_tokens_map.json
2023-02-09 17:09:00,208	ERROR trial_runner.py:1062 -- Trial HuggingFaceTrainerPatched_82a20_00000: Error processing event.
ray.exceptions.RayTaskError(PicklingError): ray::_Inner.train() (pid=74513, ip=10.0.55.91, repr=HuggingFaceTrainerPatched)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/train/_internal/utils.py", line 54, in check_for_failure
    ray.get(object_ref)
ray.exceptions.RayTaskError(PicklingError): ray::RayTrainWorker._RayTrainWorker__execute() (pid=75470, ip=10.0.55.

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 results = trainer.fit()                                                                      │
│   2                                                                                              │
│                                                                                                  │
│ /home/ray/anaconda3/lib/python3.8/site-packages/ray/train/base_trainer.py:368 in fit             │
│                                                                                                  │
│   365 │   │   try:                                                                               │
│   366 │   │   │   result = result_grid[0]                                                        │
│   367 │   │   │   if result.error:                                                               │
│ ❱ 368 │   │   │   │   raise result.error                                                         │
│   369 │   │   except TuneError as e:                                                             │
│   370 │   │   │   raise TrainingFailedError from e                                               │
│   371 │   │   return result                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RayTaskError(PicklingError): [36mray::_Inner.train()[39m (pid=74513, ip=10.0.55.91, repr=HuggingFaceTrainerPatched)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/train/_internal/utils.py", line 54, in 
check_for_failure
    ray.get(object_ref)
ray.exceptions.RayTaskError(PicklingError): [36mray::RayTrainWorker._RayTrainWorker__execute()[39m (pid=75470, 
ip=10.0.55.91, repr=<ray.train._internal.worker_group.RayTrainWorker object at 0x7f54b0efe700>)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/train/_internal/worker_group.py", line 31, in __execute
    raise skipped from exception_cause(skipped)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/train/_internal/utils.py", line 129, in 
discard_return_wrapper
    train_func(*args, **kwargs)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/train/huggingface/huggingface_trainer.py", line 417, in
_huggingface_train_loop_per_worker
    trainer.train()
  File "/mnt/cluster_storage/pypi/lib/python3.8/site-packages/transformers/trainer.py", line 1543, in train
    return inner_training_loop(
  File "/mnt/cluster_storage/pypi/lib/python3.8/site-packages/transformers/trainer.py", line 1883, in 
_inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
  File "/mnt/cluster_storage/pypi/lib/python3.8/site-packages/transformers/trainer.py", line 2135, in 
_maybe_log_save_evaluate
    self._save_checkpoint(model, trial, metrics=metrics)
  File "/mnt/cluster_storage/pypi/lib/python3.8/site-packages/transformers/trainer.py", line 2192, in 
_save_checkpoint
    self.save_model(output_dir, _internal_call=True)
  File "/mnt/cluster_storage/pypi/lib/python3.8/site-packages/transformers/trainer.py", line 2646, in save_model
    self._save(output_dir)
  File "/mnt/cluster_storage/pypi/lib/python3.8/site-packages/transformers/trainer.py", line 2728, in _save
    torch.save(self.args, os.path.join(output_dir, TRAINING_ARGS_NAME))
  File "/home/ray/anaconda3/lib/python3.8/site-packages/torch/serialization.py", line 423, in save
    _save(obj, opened_zipfile, pickle_module, pickle_protocol)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/torch/serialization.

In [ ]:
results.checkpoint

In [ ]:
from ray.air import Checkpoint
checkpoint = Checkpoint.from_directory("/mnt/cluster_storage/HuggingFaceTrainer_2023-02-08_14-55-52/HuggingFaceTrainer_bca80_00000_0_2023-02-08_14-55-53/rank_0/gpt-j-6B/checkpoint-394/")

In [ ]:
checkpoint

In [2]:
from ray.train.huggingface import HuggingFaceCheckpoint
from ray.air._internal.checkpointing import (
    load_preprocessor_from_dir,
    save_preprocessor_to_dir,
)

class HuggingFaceCheckpointPatched(HuggingFaceCheckpoint):
    def get_preprocessor(self):
        """Return the saved preprocessor, if one exists."""

        # The preprocessor will either be stored in an in-memory dict or
        # written to storage. In either case, it will use the PREPROCESSOR_KEY key.

        with self.as_directory() as checkpoint_path:
            preprocessor = load_preprocessor_from_dir(checkpoint_path)

        return preprocessor

In [ ]:
from ray.train.huggingface import HuggingFacePredictor
from transformers import set_seed

@ray.remote(num_gpus=1)
def predict(uri, seed=None):
    if seed is None:
        rng = np.random.default_rng(seed=None)
        seed = rng.integers(0, 2**16)
    print(f"seed: {seed}")
    set_seed(seed)
    checkpoint = HuggingFaceCheckpointPatched.from_uri(uri)
    print("creating predictor")
    predictor = HuggingFacePredictor.from_checkpoint(checkpoint, task="text-generation", device=0, torch_dtype=torch.bfloat16)
    # No need to use AIR preprocessor, and it looks like the one I coded has
    # issues with being loaded, so we just get rid of it
    predictor._preprocessor = None
    print("predicting")
    return predictor.predict(
        pd.DataFrame([["Romeo:"]]),
        do_sample=True, 
        max_new_tokens=256, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=3
    )

In [ ]:
prediction_tasks = [predict.remote(checkpoint.uri) for i in range(8)]
predictions = ray.get(prediction_tasks)

In [ ]:
predictions